In [101]:
import pandas as pd
import numpy as np
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from langchain_community.agent_toolkits import create_sql_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv,find_dotenv
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_experimental.tools import PythonAstREPLTool
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [23]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [3]:
df=pd.read_csv(filepath_or_buffer="data/titanic.csv")

In [4]:
df.sample(3)

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
62,0,3,Master. Harald Skoog,male,4.0,3,2,27.9000
750,1,2,Mrs. Samuel (Jane Laver) Herman,female,48.0,1,2,65.0000
492,0,3,Mr. Gerious Yousseff,male,26.0,0,0,14.4583


In [5]:
df.shape

(887, 8)

In [6]:
df.columns.tolist()

['Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'Siblings/Spouses Aboard',
 'Parents/Children Aboard',
 'Fare']

<h3> Using SQL </h3>

In [14]:
engine=create_engine(url="sqlite:///titanic.db")
df.to_sql(name="titanic",con=engine,index=False)

887

In [15]:
engine

Engine(sqlite:///titanic.db)

In [16]:
db=SQLDatabase(engine=engine)

In [17]:
db.dialect

'sqlite'

In [18]:
db.get_usable_table_names()

['titanic']

In [19]:
db.run(command="""
SELECT
    *
FROM
    titanic
WHERE 
    age<2;
""")

"[(1, 2, 'Master. Alden Gates Caldwell', 'male', 0.83, 0, 2, 29.0), (0, 3, 'Master. Eino Viljami Panula', 'male', 1.0, 4, 1, 39.6875), (1, 3, 'Miss. Eleanor Ileen Johnson', 'female', 1.0, 1, 1, 11.1333), (1, 2, 'Master. Richard F Becker', 'male', 1.0, 2, 1, 39.0), (1, 1, 'Master. Hudson Trevor Allison', 'male', 0.92, 1, 2, 151.55), (1, 3, 'Miss. Maria Nakid', 'female', 1.0, 0, 2, 15.7417), (0, 3, 'Master. Sidney Leonard Goodwin', 'male', 1.0, 5, 2, 46.9), (1, 3, 'Miss. Helene Barbara Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 3, 'Miss. Eugenie Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 2, 'Master. Viljo Hamalainen', 'male', 0.67, 1, 1, 14.5), (1, 3, 'Master. Bertram Vere Dean', 'male', 1.0, 1, 2, 20.575), (1, 3, 'Master. Assad Alexander Thomas', 'male', 0.42, 0, 1, 8.5167), (1, 2, 'Master. Andre Mallet', 'male', 1.0, 0, 2, 37.0042), (1, 2, 'Master. George Sibley Richards', 'male', 0.83, 1, 1, 18.75)]"

<h3> Creating SQL Agent for Interaction </h3>

In [124]:
# llm=ChatGoogleGenerativeAI(model="gemini-1.5-pro")
llm=ChatOpenAI(model="gpt-3.5-turbo")

In [125]:
agentExecutor=create_sql_agent(llm=llm,db=db,agent_type="tool-calling",verbose=True)

In [126]:
agentExecutor.invoke(input={"input":"What is the average age of non-survivors and survivors?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


titanic
Invoking: `sql_db_schema` with `{'table_names': 'titanic'}`



CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Parents/Children Aboard" BIGINT, 
	"Fare" FLOAT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings/Spouses Aboard	Parents/Children Aboard	Fare
0	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/
Invoking: `sql_db_query` with `{'query': 'SELECT AVG(Age) AS avg_age, Survived FROM titanic GROUP BY Survived'}`


[(30.13853211009174, 0), (28.408391812865496, 1)]The average age of non-survivors is approximately 30.14 years, and the average age of survivors is approximately 28.41 years.

> Finished chain.


{'input': 'What is the average age of non-survivors and survivors?',
 'output': 'The average age of non-survivors is approximately 30.14 years, and the average age of survivors is approximately 28.41 years.'}

<h3>Pandas</h3>

In [62]:
aiMessage=llm.invoke(input="I have a Pandas Dataframe 'df' with columns 'Age' and 'Fare'. Write a code to compute the Correlation between the two columns. Return only the Python Code snippet and nothing else.")

In [63]:
print(aiMessage.content)

```python
correlation = df['Age'].corr(df['Fare'])
```


<h4> Load our CSV Table as a Dataframe, and give the tool access to this DataFrame</h4>

In [80]:
tool=PythonAstREPLTool(locals={"df":df})

In [65]:
df['Fare'].mean()

32.30542018038331

In [66]:
tool.invoke(input="df['Fare'].mean()")

32.30542018038331

<h3> Agent </h3>

In [127]:
agent=create_pandas_dataframe_agent(llm=llm, df=df,agent_type='tool-calling',verbose=True)

In [128]:
question={"input":"What's the correlation between age and fare? Is that greater than the correlation between fare and survival?"}

In [129]:
agent.invoke(input=question)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Age'].corr(df['Fare'])"}`


0.11232863699941616
Invoking: `python_repl_ast` with `{'query': "df['Fare'].corr(df['Survived'])"}`


0.2561785496289603The correlation between age and fare is approximately 0.112, and the correlation between fare and survival is approximately 0.256. Therefore, the correlation between fare and survival is greater than the correlation between age and fare.

> Finished chain.


{'input': "What's the correlation between age and fare? Is that greater than the correlation between fare and survival?",
 'output': 'The correlation between age and fare is approximately 0.112, and the correlation between fare and survival is approximately 0.256. Therefore, the correlation between fare and survival is greater than the correlation between age and fare.'}

In [130]:
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
